In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

import yfinance as yf

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Session ID and CSRF token obtained from somewhere (e.g., through authentication)
session_id = 'jbj4ibouxarr9trfu2zndv9ns07n8qnl'
csrf_token = 'RBIDkDj3gz0wIf3EH0JGPgGdYX9tLizS'

In [3]:
# Construct headers with session ID and CSRF token
headers = {
    'Cookie': f'sessionid={session_id}',  # Assuming the session ID is passed via cookie
    'X-CSRFToken': csrf_token  # Assuming the CSRF token is passed in a header
}

# Fetch the HTML content
url = "https://www.screener.in/people/99911/graviton-research-capital-llp/"
response = requests.get(url, headers=headers)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table in the HTML
table = soup.find('table', class_='data-table')

# Extract the headers
headers = ['Symbol','Company', 'Date']

# Extract the rows
rows = []
for tr in table.find('tbody').find_all('tr'):
    company_td = tr.find('td', class_='text')
    company_name = company_td.find('a').text.strip()
    company_code = company_td.find('a').get('href').split('/')[2].upper()
    date = company_td.find('span', class_='sub').text.strip()
    rows.append([company_code, company_name, date])

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Drop Duplicates and Convert Date into Date format
df = df.drop_duplicates()
df['Date'] = pd.to_datetime(df['Date'])

In [13]:
# Filter dataframe based on date
df_filtered = df.loc[df['Date'] >= pd.to_datetime('2024-05-01')]
df_filtered = df_filtered.loc[df_filtered['Date'] <= pd.to_datetime('2024-05-07')]

In [14]:
df_filtered

In [15]:
# Define functions to download stock data, calculate indicators, plot data, and process stocks
def download_stock_data(symbol):
    return yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)

def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data

def plot_stock_data(symbol, data, company_name, date):
    # Prepare plot data
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=180))]

    # Setup subplot layout
    figure = make_subplots(specs=[[{"secondary_y": True}]])

    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'))

    # Adjust Y-axis and layout for stock prices
    figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Plot moving averages and volume
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'))
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True)
    
    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
    figure.update_yaxes(visible=False, secondary_y=True)
    
    # Add company comment and PDF link below the chart using annotations
    # figure.update_layout(
    #     annotations=[
    #         dict(
    #             text=f"<b>Company Name:</b> {company_name}<br><b>Date:</b> {date}<br>",
    #             xref="paper", yref="paper", x=0, y=-0.2, showarrow=False, align='left',
    #             xanchor='left', yanchor='top'
    #         )
    #     ],
    #     margin={'b': 200}  # Extend the bottom margin to make room for the annotation
    # )

    return figure

def process_stocks(all_symbols, comments, pdf_links):
    num_processed = 0
    file_count = 1
    while num_processed < len(all_symbols):
        figure_html = open(f'HFT-Trades{file_count}.html', 'w')
        for idx, symbol in enumerate(all_symbols[num_processed:num_processed+100]):
            try:
                data = download_stock_data(symbol)
                data = calculate_technical_indicators(data)
                comment = comments[num_processed + idx]
                pdf_link = pdf_links[num_processed + idx]
                figure = plot_stock_data(symbol, data, comment, pdf_link)
                figure_html.write(figure.to_html(full_html=False))
            except Exception as e:
                print(f"Error processing symbol {symbol}: {e}")
        figure_html.close()
        num_processed += 100
        file_count += 1

if __name__ == "__main__":
    symbols = df_filtered['Symbol'].values
    company_name = df['Company'].values
    date = df['Date'].values
    process_stocks(symbols, company_name, date)
    print('Execution Completed!!!!')